# Temperature Equation (1D)

In one dimension, the diffusive component of the *temperature evolution equation* is expressed as follows, assuming only radiogenic heat sources:

$\begin{equation}
\frac{\partial T}{\partial t} = -\frac{\partial q_x}{\partial x} + Q.
\end{equation}$ 

By incorporating Fourier’s law and allowing for spatially variable thermal properties, the equation becomes:

$\begin{equation}
\frac{\partial T}{\partial t} = \frac{\partial}{\partial x} \left( k_x \frac{\partial T}{\partial x} \right) + Q. 
\end{equation}$

Assuming constant thermal properties, Equation (2) simplifies to:

$\begin{equation}
\frac{\partial T}{\partial t} = \kappa \frac{\partial^2 T}{\partial x^2} + \frac{Q}{\rho c_p},
\end{equation}$
  
where $\kappa = k/\rho/c_p$ is the thermal diffusivity [m²/s], and $Q$ is the volumetric heat production rate [W/m³].  

Equation (3) is classified as a *parabolic partial differential equation* (PDE), which can be solved numerically given appropriate initial and boundary conditions.

## Explicit Finite Difference Scheme (FTCS; Forward Euler)

A fundamental and intuitive approach to solving the 1D heat diffusion equation is the **Forward in Time and Centered in Space (FTCS)** scheme, implemented in an **explicit** manner.

This method approximates the continuous PDE on a discrete grid and converges to the analytical solution as the spatial ($\Delta x$) and temporal ($\Delta t$) resolutions are refined. Its main advantages are **simplicity** and **computational efficiency**.

However, the FTCS scheme is **conditionally stable**. Its stability is governed by the *heat diffusion stability criterion*, which can be derived via *Von Neumann* analysis. This assesses how numerical perturbations grow or decay over time. 

In the following, we implement the FTCS scheme by combining a three-point stencil in space with a two-point stencil in time. 

For a uniform, one-dimensional grid, the stability condition is:

$\begin{equation}
\Delta t < \frac{\Delta{x^2}}{2 \kappa}.
\end{equation}$ 

Consequently, the maximum allowable time step is constrained by the spatial resolution.

Discretizing Equation (3) with the FTCS scheme gives:

$\begin{equation}
\frac{T_{i}^{n+1} - T_{i}^{n} }{\Delta t} = \kappa \frac{T_{i-1}^{n} - 2T_{i}^{n} + T_{i+1}^{n}}{\Delta{x^2}} + \frac{Q_{i}^n}{\rho c_p},
\end{equation}$ 

where 
$i$ is the spatial grid index,
$n$ is the time step index,
$\Delta x$ is the grid spacing, and
$\Delta t$ is the time step.

Solving for $T_i^{n+1}$ gives 

$\begin{equation}
T_{i}^{n+1} = T_{i}^{n} + a \left(T_{i-1}^{n} - 2T_{i}^{n} + T_{i+1}^{n} \right) + \frac{Q_{i}^n \Delta t}{\rho c_p}, 
\end{equation}$

where 

$\begin{equation} 
a = \frac{\kappa \Delta t}{\Delta x^2}.
\end{equation}$

Equation (6) is solved for all interior nodes at each time step, assuming initial and boundary conditions are specified. For implementation details, see the [source code](https://github.com/GeoSci-FFM/GeoModBox.jl/blob/main/src/HeatEquation/1Dsolvers.jl).

### Boundary Conditions 

COMMENT: BC apply to both explicit and implicit, they could be a more general section (instead of subsection in explicit)

Boundary conditions are implemented using ghost nodes located at $\frac{\Delta x}{2}$ outside the the domain boundaries. Within the `GeoModBox.jl`, the two most common thermal boundary conditions are currently considered: Dirichlet and Neumann.

**Dirichlet Boundary Condition**

The Dirichlet condition specifies a fixed temperature at the boundary. The temperature at the left (West) and right (East) ghost nodes is given by:

$\begin{equation}
T_{G,W} = 2T_{\textrm{BC},W} - T_{1},
\end{equation}$

$\begin{equation}
T_{G,E} = 2T_{\textrm{BC},E} - T_{\textrm{nc}},
\end{equation}$

where 
$T_{\textrm{BC},W}$ and $T_{\textrm{BC},E}$ are the prescribed boundary temperatures,
$T_1$ and $T_{\textrm{nc}}$ are the temperatures at the first and last interior centroids,
$T_{G,W}$ and $T_{G,E}$ are the ghost node temperatures on the west and east boundaries, and
$nc$ is the number of internal centroids.

**Neumann Boundary Condition**

The Neumann condition specifies a fixed gradient (e.g., a heat flux or temperature gradient) across the boundary. The ghost node temperatures are defined as:

$\begin{equation}
T_{G,W} = T_{1} - c_{W} \Delta{x},
\end{equation}$

$\begin{equation}
T_{G,E} = T_{\textrm{nc}} + c_{E} \Delta{x},
\end{equation}$

with:  

$\begin{equation}
\left. c_{W} = \frac{\partial{T}}{\partial{x}} \right\vert_{W},\ \textrm{and}\ \left. c_{E} = \frac{\partial{T}}{\partial{x}} \right\vert_{E}, 
\end{equation}$

where $c_W$ and $c_E$ are the prescribed temperature gradients across the west and east boundaries, respectively. These ghost node values are substituted into the discretized Equation (6) for the internal centroids adjacent to the boundaries. Using ghost nodes consistently enforces the boundary conditions at each time step and preserves the second-order accuracy treatment of the three-point stencil (e.g. Duretz et al., 2011).

## Implicit Scheme (Backward Euler)

The fully implicit finite difference scheme, also known as the **Backward Euler** method, is **unconditionally stable**, allowing time steps larger than those permitted by the CFL criterion.

> **Note**: While the implicit method is unconditionally stable, very large time steps may still yield inaccurate results, particularly for resolving small-scale thermal gradients (THIS STATEMENT IS NOT CLEAR: which scale? spatial scale? does running with poor time resolution strongly afftec spatial resolution?).

In 1D and based on a three-point stencil, the discretized temperature evolution equation becomes:

$\begin{equation}
\frac{T_{i}^{n+1}-T_{i}^n}{\Delta t} = \kappa \frac{T_{i-1}^{n+1}-2T_{i}^{n+1}+T_{i+1}^{n+1}}{\Delta{x^2}} + \frac{Q^n}{\rho c_p},
\end{equation}$

where $i$ is the spatial index, $n$ is the time step index, $\Delta t$ is the time step, and $\Delta x$ is the spatial grid spacing. Rearranging the equation into known (right-hand side) and unknown (left-hand side) terms yields a system of equations considering all internal centroids:

$\begin{equation}
-a T_{i-1}^{n+1} + \left(2a + b \right) T_{i}^{n+1} - a T_{i+1}^{n+1} = b T_{i}^n + \frac{Q^n}{\rho c_p},
\end{equation}$

with 

$\begin{equation}\begin{split}
a & = \dfrac{\kappa}{\Delta x^2}, \\
b & = \dfrac{1}{\Delta t}.
\end{split}\end{equation}$

These equations are a tridiagonal system of equations in the form of:

$\begin{equation}
\mathbf{K} \cdot \bm{x} = \bm{b}
\end{equation}$

where $\mathbf{K}$ is the coefficient matrix (with three non-zero diagonals), $\bm{x}$ is the unknown solution vector, that is the temperature at time step $n+1$, and $\bm{b}$ is the known right-hand side.

### General Solution

A general approach to solve this system of equations is the **defection correction**. The diffusive part of the temperature equation is reformulated by introducing a residual term $\bm{r}$, which quantifies the deviation from the true solution and can be reduced iteratively to improve accuracy through successive correction steps. In implicit form, Equation (16) can be rewritten as:

$\begin{equation}
\mathbf{K} \cdot \bm{x} - \bm{b} = \bm{r}, 
\end{equation}$

where $\bm{r}$ is the residual (or defect). The coefficients of the matrix can be calculated by the Jacobian of $\bm{r}$ via: 

$\begin{equation}
{K_{ij}}=\frac{\partial{{r}_i}}{\partial{{x}_j}}.
\end{equation}$

COMMENT: not sure it this is neede here because the coefficients are already calculated above. If we make this definition here, it will be confuing for the reader. 

We can solve for the unknown vector $\bm{x}$ by assuming an initial temperature guess, calculating its residual, and performing a correction of the temperature. For non-linear problems, the process is repeated iteratively until the residual is sufficiently small. In the linear case, one iteration yields the exact solution.

Given an initial temperature guess $\bm{T}^k$, the initial residual is:

$\begin{equation}
\bm{r}^k = \mathbf{K} \cdot \bm{T}^k - \bm{b}.
\end{equation}$

To reduce the residual, a correction term $\delta \bm{T}$ is defined such that:

$\begin{equation}
0 = \bm{K} \left(\bm{T}^k + \delta \bm{T} \right) - \bm{b} = \bm{K} \bm{T}^k - \bm{b} + \bm{K} \delta \bm{T} = \bm{r}^k + \bm{K} \delta \bm{T}.
\end{equation}$

Rearranging gives: 

$\begin{equation}
\bm{r}^k = -\mathbf{K} \delta \bm{T}, 
\end{equation}$

and hence: 

$\begin{equation}
\delta \bm{T} = -\mathbf{K}^{-1} \bm{r}^k. 
\end{equation}$

Thus, the updated solution becomes: 

$\begin{equation}
\bm{T}^{k+1} = \bm{T}^k + \delta \bm{T},
\end{equation}$

where $\bm{T}^{k+1}$ is the updated temperature after one iteration step. 

Within `GeoModBox.jl` the residual $\bm{r}$ is calculated on the internal centroids using the extended temperature field including the ghost nodes of the current time step as an initial guess: 

$\begin{equation}
\frac{\partial{T}}{\partial{t}} - \kappa \frac{\partial^2{T}}{\partial{x^2}} - \frac{Q}{\rho c_p} = r,  
\end{equation}$

and in discretized, implicit finit-difference form: 

$\begin{equation}
\frac{T_{i,ext}^{n+1} - T_{i,ext}^{n}}{\Delta{t}} - \kappa \frac{T_{\textrm{ext},i-1}^{n+1} - 2 T_{\textrm{ext},i}^{n+1} + T_{\textrm{ext},i+1}^{n+1}}{\Delta{x^2}} - \frac{Q^n}{\rho c_p}= r_j, 
\end{equation}$

where $i=2:(nc+1)$ is the index of the internal centroid of the extended temperature field and $j=1:nc$ is the index of the internal centroids of the regular field. Rewriting Equation (25) and substituting the coefficients using Equation (15) results in: 

$\begin{equation}
bT_{\textrm{ext},i}^{n+1}
-bT_{\textrm{ext},i}^n
-aT_{\textrm{ext},i-1}^{n+1}
+2aT_{\textrm{ext},i}^{n+1}
-aT_{\textrm{ext},i+1}^{n+1}
-\frac{Q^n}{\rho c_p}=r_j,
\end{equation}$

and further summarizing to: 

$\begin{equation}
-aT_{\textrm{ext},i-1}^{n+1}
+\left(2a+b\right)T_{\textrm{ext},i}^{n+1}
-aT_{\textrm{ext},i+1}^{n+1}
-bT_{\textrm{ext},i}^n
-\frac{Q^n}{\rho c_p}=r_j,
\end{equation}$

which corresponds to the matrix form of Equation (17), where $T_{\textrm{ext},i}^{n+1}$ is the unknown vector $x$,$-bT_{\textrm{ext},i}^n -\frac{Q^n}{\rho c_p}$ is the known vector $b$, and $-a$ and $2a+b$ are the coefficients of the non-zero diagonals of the coefficient matrix. With the residual vector $R$ and the coefficient matrix $\bm{K}$ one can calculate the correction term for the temperature via Equation (22). 

### Boundary Condition 

The boundary conditions are implemented again using the temperatures at the ghost nodes which are defined as before (see Equations (8)-(11)). The ghost nodes enables the calculation of the residual $R_i$ for the internal centroids adjacent to the boundaries. To maintain symmetry in the coefficient matrix, however, the matrix coefficients must be modified for the nodes adjacent to the boundaries.

**Dirichlet Boundary Condition**

The equations for the internal centroids adjacent to the boundary are then given by: 

**West Boundary**

$\begin{equation}
-aT_{G,W}
+\left(2a+b\right)T_{\textrm{ext},2}^{n+1}
-aT_{\textrm{ext},3}^{n+1}
-bT_{\textrm{ext},2}^n
-\frac{Q^n}{\rho c_p}=r_1,
\end{equation}$

$\begin{equation}
-a\left(2T_{\textrm{BC},W}-T_{\textrm{ext},2}^{n+1}\right)
+\left(2a+b\right)T_{\textrm{ext},2}^{n+1}
-aT_{\textrm{ext},3}^{n+1}
-bT_{\textrm{ext},2}^n
-\frac{Q^n}{\rho c_p}=r_1,
\end{equation}$

$\begin{equation}
\left(3a+b\right)T_{\textrm{ext},2}^{n+1}
-aT_{\textrm{ext},3}^{n+1}
-bT_{\textrm{ext},2}^n
-2aT_{\textrm{BC},W}
-\frac{Q^n}{\rho c_p}=r_1,
\end{equation}$

**East Boundary**

$\begin{equation}
-aT_{\textrm{ext},nc}^{n+1}
+\left(2a+b\right)T_{\textrm{ext},nc+1}^{n+1}
-aT_{G,E}
-bT_{\textrm{ext},nc+1}^n
-\frac{Q^n}{\rho c_p}=r_{\textrm{nc}},
\end{equation}$

$\begin{equation}
-aT_{\textrm{ext},nc}^{n+1}
+\left(2a+b\right)T_{\textrm{ext},nc+1}^{n+1}
-a\left(2T_{\textrm{BC},E}-T_{\textrm{ext},nc+1}^{n+1}\right)
-bT_{\textrm{ext},nc+1}^n
-\frac{Q^n}{\rho c_p}=r_{\textrm{nc}},
\end{equation}$

$\begin{equation}
-aT_{\textrm{ext},nc}^{n+1}
+\left(3a+b\right)T_{\textrm{ext},nc+1}^{n+1}
-bT_{\textrm{ext},nc+1}^n
-2aT_{\textrm{BC},E}
-\frac{Q^n}{\rho c_p}=r_{\textrm{nc}},
\end{equation}$

**Neumann Boundary Conditions**

**West Boundary**

$\begin{equation}
-aT_{G,W}
+\left(2a+b\right)T_{\textrm{ext},2}^{n+1}
-aT_{\textrm{ext},3}^{n+1}
-bT_{\textrm{ext},2}^n
-\frac{Q^n}{\rho c_p}=r_1, 
\end{equation}$

$\begin{equation}
-a\left(T_{\textrm{ext},2}^{n+1} - c_{W} \Delta{x}\right)
+\left(2a+b\right)T_{\textrm{ext},2}^{n+1}
-aT_{\textrm{ext},3}^{n+1}
-bT_{\textrm{ext},2}^n
-\frac{Q^n}{\rho c_p}=r_1, 
\end{equation}$

$\begin{equation}
+\left(a+b\right)T_{\textrm{ext},2}^{n+1}
-aT_{\textrm{ext},3}^{n+1}
-bT_{\textrm{ext},2}^n
+a c_{W} \Delta{x}
-\frac{Q^n}{\rho c_p}=r_1, 
\end{equation}$

**East Boundary**

$\begin{equation}
-aT_{\textrm{ext},nc}^{n+1}
+\left(2a+b\right)T_{\textrm{ext},nc+1}^{n+1}
-aT_{G,E}
-bT_{\textrm{ext},nc+1}^n
-\frac{Q^n}{\rho c_p}=r_{\textrm{nc}},
\end{equation}$

$\begin{equation}
-aT_{\textrm{ext},nc}^{n+1}
+\left(2a+b\right)T_{\textrm{ext},nc+1}^{n+1}
-a\left(T_{\textrm{ext},nc+1}^{n+1} + c_{E} \Delta{x}\right)
-bT_{\textrm{ext},nc+1}^n
-\frac{Q^n}{\rho c_p}=r_{\textrm{nc}},
\end{equation}$

$\begin{equation}
-aT_{\textrm{ext},nc}^{n+1}
+\left(a+b\right)T_{\textrm{ext},nc+1}^{n+1}
-bT_{\textrm{ext},nc+1}^n
-ac_{E} \Delta{x}
-\frac{Q^n}{\rho c_p}=r_{\textrm{nc}},
\end{equation}$

where 
$T_{\textrm{BC},W}$, $T_{\textrm{BC},E}$ are the prescribed boundary temperatures,
$c_W$, $c_E$ are the prescribed temperature gradients at the west and east boundaries, and
$nc$ is the number of internal centroids. 

These adjustments ensure that the boundary conditions are enforced consistently while preserving the symmetry of the implicit solver.

### Special Case - A Linear Problem 

If the problem is linear and the exact solution is reached within one single iteration step, the system of equations is reduced to Equation (14). Thus, one can solve the system of equations directly via a *left matrix division*: 

$\begin{equation}
\bf{x} = \mathbf{A}^{-1} \bm{b}. 
\end{equation}$

The coefficient matrix remains the same, even for the given boundary conditions. However, the right hand side needs to be updated accordingly (simply setting $\bm{r}=0$ and add the know parameters to the right hand side of the equations). Thus, the equations for the internal centroids adjacent to the boundaries are defined as: 

**Dirichlet Boundary Condition**

**West boundary**

$\begin{equation}
\left(3 a + b\right) T_{1}^{n+1} - a T_{2}^{n+1} = b T_{1}^{n} + 2 a T_{\textrm{BC},W},
\end{equation}$

**East boundary**

$\begin{equation}
-a T_{\textrm{nc}-1}^{n+1} + \left(3 a + b\right) T_{\textrm{nc}}^{n+1}  = b T_{\textrm{nc}}^{n} + 2 a T_{\textrm{BC},E}, 
\end{equation}$

**Neumann Boundary Condition**

**West boundary**

$\begin{equation}
\left(a + b\right) T_{1}^{n+1} - a T_{2}^{n+1} = b T_{1}^{n} - a c_{W} \Delta{x},
\end{equation}$

**East boundary**

$\begin{equation}
-a T_{\textrm{nc}-1}^{n+1} + \left(a + b\right) T_{\textrm{nc}}^{n+1}  = b T_{\textrm{nc}}^{n} + a c_{E} \Delta{x}. 
\end{equation}$

